# Cleaning the Data

## Import the Libraries

In [13]:
import pandas as pd
import numpy as np

# Set the max columns to max because the dataset has a lot of columns
pd.set_option('display.max_columns', None)

pd.set_option("display.max_rows", None)

## Load the Data

In [3]:
import json

def load_data(title):
  with open(title, encoding='utf-8') as f:
    return json.load(f)

In [14]:
pop_df = load_data('skyrim_population.json')
pop_df[4]

{'Source': 'https://en.uesp.net/wiki/Skyrim:Babette',
 'Name': 'Babette',
 'Location': 'Dark Brotherhood Sanctuary, Dawnstar Sanctuary',
 'Race': 'Breton',
 'Gender': 'Female',
 'Level': '10',
 'Class': 'Assassin',
 'RefID': '0001D4BC',
 'BaseID': '0001D4B7',
 'Services': True,
 'Training': 'Alchemy (Master)',
 'Merchant': True,
 'Gold': '500 (+1000 Master Trader)',
 'Sells': 'See Standard Merchandise',
 'Buys': 'Animal Parts, Food, Ingredients, Poisons, Potions, Raw Food, Recipes, Tools',
 'Other Information': True,
 'Health': '125',
 'Magicka': '15',
 'Stamina': '95',
 'Primary Skills': 'Archery, One-handed, Sneak, Conjuration, Light Armor',
 'Race Details': 'Vampire, Undead, Child',
 'Class Details': 'TrainerAlchemyMaster',
 'Morality': 'No Crime',
 'Aggression': 'Unaggressive',
 'Essential': 'Yes',
 'Faction(s)': "Alchemy Trainer; Babette's Bed Ownership; Babette's Dark Brotherhood Services; The Dark Brotherhood; FavorExcludedFaction; JobApothecaryFaction; JobMerchantFaction; JobMi

In [16]:
# Convert the data into dataframe and save it as csv
pop_df = pd.DataFrame(pop_df)
pop_df.to_csv('skyrim_population_raw_2.csv', index=False)

In [17]:
df = pd.read_csv('skyrim_population_raw_2.csv')

## Inspect the Data

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1051 entries, 0 to 1050
Data columns (total 47 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Source             1051 non-null   object
 1   Name               1051 non-null   object
 2   Home City          540 non-null    object
 3   House              258 non-null    object
 4   Race               1038 non-null   object
 5   Gender             1038 non-null   object
 6   Level              1051 non-null   object
 7   Class              1038 non-null   object
 8   RefID              1014 non-null   object
 9   BaseID             1044 non-null   object
 10  Other Information  1051 non-null   bool  
 11  Health             1034 non-null   object
 12  Magicka            1032 non-null   object
 13  Stamina            1018 non-null   object
 14  Primary Skills     911 non-null    object
 15  Morality           1017 non-null   object
 16  Aggression         1017 non-null   object


In [19]:
pop_df.isnull().sum().sort_values(ascending=False)

Available            1049
Added by             1046
Horses               1045
Merchant §           1041
Merchant ‡           1040
Type                 1038
Soul                 1038
Species              1038
AdoptHF              1028
Stronghold           1015
StewardHF            1014
Respawns              995
Training              990
Race Details          986
Follower              984
Services              982
Marry                 980
Home Town             936
Store                 935
Protected             931
Perks                 930
Merchant              913
Buys                  905
Gold                  903
Sells                 892
Added by              875
House                 793
Essential             765
Class Details         658
Home City             511
Location              378
Primary Skills        140
Faction(s)             41
Aggression             34
Morality               34
Stamina                33
Health                 16
Magicka                16
Class       

## To Do:
1. Delete unnecessary column
2. Delete duplicated Rows
3. Fix Column Name
4. Merge information from columns like Merchant with its other variants
5. Remove refID from Name column
6. Convert columns such as Level, Health, Magicka, and Stamina to numerical value 
7. Deal with the Null values

In [86]:
df = pop_df.copy()

### Delete Unnecessary Column

In [87]:
# Delete Other Information Column
df.drop(columns=['Other Information'], inplace=True)
df.drop(columns=['Soul'], inplace=True)

### Delete Duplicated Rows

In [88]:
len(df.loc[df.duplicated(subset=['Source'])])

42

In [89]:
df.drop_duplicates(keep='first', inplace=True)

### Fix Columns Name

In [90]:
for i, col in enumerate(df.columns):
    print(i, col)

0 Source
1 Name
2 Home City
3 House
4 Race
5 Gender
6 Level
7 Class
8 RefID
9 BaseID
10 Health
11 Magicka
12 Stamina
13 Primary Skills
14 Morality
15 Aggression
16 Faction(s)
17 Class Details
18 Essential
19 Location
20 Services
21 Training
22 Merchant
23 Gold
24 Sells
25 Buys
26 Race Details
27 Follower
28 Respawns
29 Store
30 Added by
31 AdoptHF
32 Protected
33 Perks
34 Marry
35 StewardHF
36 Merchant §
37 Merchant ‡
38 Available
39 Species
40 Type
41 Horses
42 Home Town
43 Added by
44 Stronghold


In [92]:
df.columns[30]

'Added\xa0by'

In [93]:
df.rename(columns={
    'Home\xa0City': 'Home City',
    'Added\xa0by': 'Added_by',
    'Home\xa0Town': 'Home Town'}, inplace=True)

### Merge Values from Other Columns

In [94]:
# Merchant column and its variants
df['Merchant'].fillna(df['Merchant §'], inplace=True)
df['Merchant'].fillna(df['Merchant ‡'], inplace=True)

df.drop(columns=['Merchant §', 'Merchant ‡'], inplace=True)

In [95]:
# Added by column
df['Added by'].fillna(df['Added_by'], inplace=True)
df.drop(columns=['Added_by'], inplace=True)

In [96]:
# Fillna the Home City with values from Home Town and Stronghold
df['Home City'].fillna(df['Home Town'], inplace=True)
df['Home City'].fillna(df['Stronghold'], inplace=True)
df.drop(columns=['Home Town'], inplace=True)
df.drop(columns=['Stronghold'], inplace=True)

In [97]:
# Fillna Race with type of several characters that consists of animals and skeletons
df['Race'].fillna(df['Type'], inplace=True)
df.loc[df['Race'] == '', 'Race'] = 'Undead'
df.drop(columns=['Type', 'Species'], inplace=True)

### Remove refID from Name column

In [99]:
df.reset_index(inplace=True, drop=True)

In [100]:
df.loc[df['Name'].str.contains('Ref')]

,Source,Name,Home City,House,Race,Gender,Level,Class,RefID,BaseID,Health,Magicka,Stamina,Primary Skills,Morality,Aggression,Faction(s),Class Details,Essential,Location,Services,Training,Merchant,Gold,Sells,Buys,Race Details,Follower,Respawns,Store,AdoptHF,Protected,Perks,Marry,StewardHF,Available,Horses,Added by
92,https://en.uesp.net/wiki/Skyrim:Torom,Torom (RefID: 000661A4),NaN,NaN,Animals,NaN,2,NaN,000661A4,0002F442,21,0,24,Sneak,NaN,NaN,LeftHandDaighresHouseFaction,NaN,NaN,Left Hand Mine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104,https://en.uesp.net/wiki/Skyrim:Vigilance,Vigilance (RefID: 0009A7AB),Markarth,Markarth Stables,Animals,NaN,PC×1 (range=6-50),NaN,0009A7AB,0009A7AA,10+(PC-1)×10,0,20+(PC-1)×5,"Block, Sneak, Archery, One-handed",NaN,NaN,MarkarthStablesFaction; PlayerPotentialAnimalF...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193,https://en.uesp.net/wiki/Skyrim:Gratian_Caerel...,Gratian Caerellius (RefID: xx020A2E),Raven Rock,NaN,Undead,NaN,1,NaN,xx020A2E,xx020A2D,0 (Dead),,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Raven Rock Mine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
195,https://en.uesp.net/wiki/Skyrim:Millius,Millius (RefID: xx020A48),Raven Rock,NaN,Undead,NaN,1,NaN,xx020A48,xx020A47,0 (Dead),,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Raven Rock Mine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
270,https://en.uesp.net/wiki/Skyrim:Snippy,Snippy (RefID: xx033A4D),NaN,NaN,Animals,NaN,1,NaN,xx033A4D,xx033A47,5,4,25,NaN,NaN,NaN,Riften Fishery Faction; TownRiftenFaction,NaN,Yes,Riften Docks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fishing
503,https://en.uesp.net/wiki/Skyrim:Tiber,Tiber (RefID: 00023EF2),Windhelm,Hollyfrost Farm,Animals,NaN,2,NaN,00023EF2,00023EF1,21,0,24,Sneak,NaN,NaN,Creature Faction; DogFaction; Hollyfrost Farm ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
504,https://en.uesp.net/wiki/Skyrim:Ysgramor_(dog),Ysgramor (RefID: 00023EEE),Windhelm,Hollyfrost Farm,Animals,NaN,2,NaN,00023EEE,00023EED,21,0,24,Sneak,NaN,NaN,Creature Faction; DogFaction; Hollyfrost Farm ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
632,https://en.uesp.net/wiki/Skyrim:Stump,Stump (RefID: 0001E62B),NaN,NaN,Animals,NaN,2,NaN,0001E62B,0001E62A,21,0,24,Sneak,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
715,https://en.uesp.net/wiki/Skyrim:Maximian_Axius,Maximian Axius (RefID: xx030C9F),NaN,NaN,Undead,NaN,1,NaN,xx030C9F,xx030C9E,0 (Dead),,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fort Frostmoth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
735,https://en.uesp.net/wiki/Skyrim:Bran,Bran (RefID: xx01AA7B),NaN,NaN,Animals,NaN,PC×1 (range=10-25),NaN,xx01AA7B,xx01AA74,100+(PC-1)×10,0,100+(PC-1)×5,"Block, Sneak, Archery, One-handed",NaN,NaN,Creature Faction; CrimeFactionRift; DLC1Hunter...,NaN,NaN,Fort Dawnguard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,Dawnguard


In [101]:
for i, row in enumerate(df['Name']):
    if row.find('Ref') != -1:
        row = row.split(' (')[0]
        df.at[i, 'Name'] = row

### Convert Level, Health, Stamina, and Magicka Column Value to Numerical

In [103]:
# Converting the Level column
df['Level (PC=10)'] = df['Level']

In [104]:
for i, val in enumerate(df['Level (PC=10)']):
    if val.find('PC') != -1:
        val = val.replace('PC', '10').replace('×', '*').replace('x', '*')
        if (val.find('range') != -1) | (val.find('Range') != -1): # If it has PC and Range 
            val, rang = val.split(' ')
            rang = rang.split('=')[1].replace(')', '')
            low, high = pd.to_numeric(rang.split('-'))
            val = eval(val)
            if val <= low:
                val = low
            elif val >= high:
                val = high
            df.at[i, 'Level (PC=10)'] = val
        elif val.find('ma*') != -1:
            val, high = val.split(' ')
            high = int(high.split('=')[1].replace(')', ''))
            val = eval(val)
            if val >= high:
                val = high
            df.at[i, 'Level (PC=10)'] = val
        elif val.find('min') != -1:
            val, low = val.split(' ')
            low = int(low.split('=')[1].replace(')', ''))
            val = eval(val)
            if val <= low:
                val = low
            df.at[i, 'Level (PC=10)'] = val
        else:
            val = eval(val)
            df.at[i, 'Level (PC=10)'] = val
    elif val.find('Radiant') != -1:
        val = val.split(' ')[1].replace('(', '').replace(')', '')
        low, high = pd.to_numeric(val.split('-'))
        val = (high + low)/2
        df.at[i, 'Level (PC=10)'] = val
    elif val.find('Leveled') != -1:
        val = val.split(' ')[1].replace('(', '').replace(')', '')
        low, high = pd.to_numeric(val.split('-'))
        val = (high + low)/2
        df.at[i, 'Level (PC=10)'] = val
    else:
        try:
            val = pd.to_numeric(val)
            df.at[i, 'Level (PC=10)'] = val
        except:
            low, high = pd.to_numeric(val.split('-'))
            val = (high + low)/2
            df.at[i, 'Level (PC=10)'] = val

In [105]:
# Solving Health, Magicka, and Stamina columns

df['Health (PC=10)'] = df['Health']
df['Magicka (PC=10)'] = df['Magicka']
df['Stamina (PC=10)'] = df['Stamina']

columns = ['Health (PC=10)', 'Magicka (PC=10)', 'Stamina (PC=10)']
for col in columns:
    for i, val in enumerate(df[col]):
        try:
            if val.find('(PC') != -1:
                val = val.replace('PC', '10').replace('×', '*').replace('x', '*')
                val = eval(val)
                df.at[i, col] = val
            elif val.find('(NPC') != -1:
                level = str(df.at[i, 'Level (PC=10)'])
                val = val.replace('NPC', level).replace('×', '*').replace('x', '*')
                val = eval(val)
                df.at[i, col] = val
            elif val.find('level') != -1:
                level = str(df.at[i, 'Level (PC=10)'])
                val = val.replace('level', level).replace('⅓', '*(1/3)').replace('⅔' ,'*(2/3)').replace('×', '*').replace('x', '*')
                val = eval(val)
                df.at[i, col] = val
            elif val.find('Radiant') != -1:
                val = val.split(' ')[1].replace('(', '').replace(')', '')
                low, high = pd.to_numeric(val.split('-'))
                val = (high + low)/2
                df.at[i, col] = val
            elif val.find('(Dead)') != -1:
                val = 0
                df.at[i, col] = val
            else:
                try:
                    val = pd.to_numeric(val)
                    df.at[i, col] = val
                except:
                    low, high = pd.to_numeric(val.split('-'))
                    val = (high + low)/2
                    df.at[i, col] = val
        except:
            continue

### Deal with the Null Values

In [123]:
df.isnull().sum().sort_values(ascending=False)

Horses             1003
AdoptHF             986
StewardHF           975
Respawns            957
Training            950
Follower            948
Race Details        946
Services            942
Marry               941
Perks               898
Protected           894
Store               893
Buys                864
Gold                862
Sells               851
Merchant            846
Added by            834
Essential           753
House               752
Class Details       633
Location            374
Home City           344
Primary Skills      132
Faction(s)           40
Morality             33
Aggression           33
Stamina (PC=10)      31
Stamina              31
Magicka (PC=10)      17
Health (PC=10)       15
Magicka              14
Health               14
Class                13
Gender               13
Name                  0
BaseID                0
RefID                 0
Level                 0
Race                  0
Level (PC=10)         0
Source                0
dtype: int64

In [111]:
# Fillna for Horse Merchant
df.loc[df['Horses'].notnull()][['Name', 'Merchant', 'Horses']]

,Name,Merchant,Horses
102,Cedran,NaN,Black and white; 1000 gold
276,Hofgrir Horse-Crusher,NaN,Grey; 1000 gold
377,Geimund,NaN,Palomino; 1000 gold
473,Skulvar Sable-Hilt,NaN,Black; 1000 gold
543,Ulundil,NaN,Bay and white; 1000 gold
910,Agrane Peryval,True,Reindeer; 500+ gold


In [119]:
df.loc[df['Horses'].notnull(), 'Merchant'] = True

In [122]:
# Delete Avalilabe column
df.drop(columns=['Available'], inplace=True)

In [141]:
df['Morality'].unique()

array(['No Crime', nan, 'Any Crime', 'Violence Against Enemies',
       'Property Crime Only'], dtype=object)

In [144]:
df.loc[df['Class'].isnull()][['Name', 'Race', 'Class', 'Gender']]

,Name,Race,Class,Gender
92,Torom,Animals,NaN,NaN
104,Vigilance,Animals,NaN,NaN
193,Gratian Caerellius,Undead,NaN,NaN
195,Millius,Undead,NaN,NaN
270,Snippy,Animals,NaN,NaN
503,Tiber,Animals,NaN,NaN
504,Ysgramor,Animals,NaN,NaN
632,Stump,Animals,NaN,NaN
715,Maximian Axius,Undead,NaN,NaN
735,Bran,Animals,NaN,NaN


In [147]:
df.drop(columns=['Services'], inplace=True)

fill_no = ['Essential', 'AdoptHF', 'StewardHF', 'Respawns', 'Follower', 'Marry', 'Protected']
fill_none = ['Training', 'Store', 'Buys', 'Sells', 'Class Details']
fill_indeterminate = ['Morality', 'Aggression', 'Class', 'Gender']

df['Added by'].fillna('Vanilla', inplace=True)

for col in fill_no:
    df[col].fillna('No', inplace=True)

for col in fill_none:
    df[col].fillna('None', inplace=True)
    
for col in fill_indeterminate:
    df[col].fillna('Indeterminate', inplace=True)

In [148]:
df.isnull().sum().sort_values(ascending=False)

Horses             1003
Race Details        946
Perks               898
Gold                862
Merchant            846
House               752
Location            374
Home City           344
Primary Skills      132
Faction(s)           40
Stamina (PC=10)      31
Stamina              31
Magicka (PC=10)      17
Health (PC=10)       15
Magicka              14
Health               14
Marry                 0
StewardHF             0
Added by              0
Protected             0
AdoptHF               0
Level (PC=10)         0
Store                 0
Respawns              0
Follower              0
Source                0
Buys                  0
Sells                 0
Name                  0
Essential             0
Class Details         0
Aggression            0
Morality              0
BaseID                0
RefID                 0
Class                 0
Level                 0
Gender                0
Race                  0
Training              0
dtype: int64

In [149]:
df.to_csv('Skyrim_Named_Characters.csv', index=False)